In [2]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df
import random
import csv

In [3]:
%run MODEL_LIST.ipynb            #Load the models

In [4]:
df = pd.read_csv("../data/ufc-master.csv")

In [5]:
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1
0,Tyron Woodley,Gilbert Burns,-170.0,150.0,58.823529,150.000000,5/30/2020,"Las Vegas, Nevada, USA",USA,Blue,...,-3.0,0.0,2.54,-7.62,-5.0,0.63,0.2,1.18,1.0,1.0
1,Blagoy Ivanov,Augusto Sakai,-105.0,-115.0,95.238095,86.956522,5/30/2020,"Las Vegas, Nevada, USA",USA,Blue,...,2.0,0.0,10.16,10.16,-4.0,2.42,-0.2,-0.83,1.0,1.0
2,Billy Quarantillo,Spike Carlyle,-130.0,110.0,76.923077,110.000000,5/30/2020,"Las Vegas, Nevada, USA",USA,Red,...,1.0,-1.0,-5.08,2.54,-4.0,3.35,-4.0,-0.80,1.0,1.0
3,Roosevelt Roberts,Brok Weaver,-360.0,300.0,27.777778,300.000000,5/30/2020,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-1.0,-5.08,0.00,2.0,2.70,0.3,-1.84,1.0,1.0
4,Mackenzie Dern,Hannah Cifers,-420.0,335.0,23.809524,335.000000,5/30/2020,"Las Vegas, Nevada, USA",USA,Red,...,0.0,-1.0,-7.62,-2.54,0.0,1.58,-0.6,-0.14,1.0,1.0


In [6]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [7]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [8]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [9]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[200:]
odds_train = odds_df[200:]
label_train = label_df[200:]

df_test = df[:200]
odds_test = odds_df[:200]
label_test = label_df[:200]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
4011
4011
4011


In [10]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
201,Volkan Oezdemir,Ilir Latifi,-190.0,165.0,52.631579,165.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,2.0,-10.16,-5.08,-7.0,-2.590000,0.500000,1.460000,0.0,1.0,0
202,Rodolfo Vieira,Oskar Piechota,-225.0,185.0,44.444444,185.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,0.0,0.00,7.62,0.0,1.280000,0.300000,-4.010000,0.0,1.0,0
203,Enrique Barzola,Bobby Moffett,-165.0,145.0,60.606061,145.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,1.0,7.62,7.62,1.0,-0.520000,1.200000,-1.820000,0.0,1.0,0
204,Gilbert Burns,Aleksei Kunchenko,120.0,-140.0,120.000000,71.428571,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-4.0,-5.08,-2.54,-2.0,0.530000,-0.700000,-2.100000,0.0,1.0,0
205,Ciryl Gane,Raphael Pessoa Nunes,-470.0,375.0,21.276596,375.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,0.0,-2.54,-7.62,-1.0,-2.720000,-0.900000,-1.320000,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,Duane Ludwig,Darren Elkins,-155.0,135.0,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0.0,0.00,2.54,6.0,-13.666667,0.000000,0.000000,0.0,1.0,1
4280,John Howard,Daniel Roberts,-210.0,175.0,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,7.62,7.62,-2.0,-18.000000,-1.000000,-4.666667,0.0,1.0,0
4281,Brendan Schaub,Chase Gormley,-260.0,220.0,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,-2.54,-2.12,0.0,-4.000000,1.000000,1.000000,0.0,1.0,0
4282,Mike Pierce,Julio Paulino,-420.0,335.0,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0.0,10.16,7.62,-5.0,-40.500000,0.000000,-3.500000,0.0,1.0,0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,label
0,Tyron Woodley,Gilbert Burns,-170.0,150.0,58.823529,150.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,2.54,-7.62,-5.0,0.63,0.2,1.18,1.0,1.0,1
1,Blagoy Ivanov,Augusto Sakai,-105.0,-115.0,95.238095,86.956522,2020-05-30,"Las Vegas, Nevada, USA",USA,Blue,...,0.0,10.16,10.16,-4.0,2.42,-0.2,-0.83,1.0,1.0,1
2,Billy Quarantillo,Spike Carlyle,-130.0,110.0,76.923077,110.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,2.54,-4.0,3.35,-4.0,-0.80,1.0,1.0,0
3,Roosevelt Roberts,Brok Weaver,-360.0,300.0,27.777778,300.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-5.08,0.00,2.0,2.70,0.3,-1.84,1.0,1.0,0
4,Mackenzie Dern,Hannah Cifers,-420.0,335.0,23.809524,335.000000,2020-05-30,"Las Vegas, Nevada, USA",USA,Red,...,-1.0,-7.62,-2.54,0.0,1.58,-0.6,-0.14,1.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Andre Ewell,Jonathan Martinez,-133.0,113.0,75.187970,113.000000,2020-02-08,"Houston, Texas, USA",USA,Red,...,0.0,0.00,-12.70,7.0,0.10,0.0,0.42,0.0,1.0,0
197,Austin Lingo,Youssef Zalel,-210.0,175.0,47.619048,175.000000,2020-02-08,"Houston, Texas, USA",USA,Blue,...,0.0,0.00,5.08,2.0,0.00,0.0,0.00,0.0,1.0,1
198,Valentina Shevchenko,Liz Carmouche,-1100.0,700.0,9.090909,700.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-1.0,2.54,0.00,-4.0,-0.24,-0.1,0.63,0.0,1.0,0
199,Vincente Luque,Mike Perry,-250.0,210.0,40.000000,210.000000,2019-08-10,"Montevideo, Uruguay",Uruguay,Red,...,-3.0,-2.54,-10.16,0.0,-0.91,-1.0,-0.21,0.0,1.0,0


In [11]:
def get_ev(input_df, input_model, input_features, input_labels, odds_input, min_ev = 0, verbose=False, get_total=False):
    df_sel = input_df[input_features]
    df_sel = df_sel.dropna()
    df_sel = pd.get_dummies(df_sel)
    labels_sel = input_labels[input_labels.index.isin(df_sel.index)]
    odds_sel = odds_input[odds_input.index.isin(df_sel.index)] 
    best_score = custom_cv_eval(df_sel, input_model, labels_sel, odds_sel, min_ev = min_ev, verbose=verbose, 
                                get_total=get_total)
    return best_score

In [12]:
def evaluate_model(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
                  test_odds, verbose=True):
    model_score = 0
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    
    
    
    display(df_train.shape)
    display(labels_train.shape)
    display(odds_train.shape)
    display(df_test.shape)
    display(labels_test.shape)
    display(odds_test.shape)
    
    input_model.fit(df_train, labels_train)

    
    
    probs = input_model.predict_proba(df_test)

    
    odds_test = np.array(odds_test)    
    
    
    prepped_test = list(zip(odds_test[:, -2], odds_test[:, -1], probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    display(ev_prepped_df)
    
    #display(df_test)
    #display(df_test)
    model_score = get_ev_from_df(ev_prepped_df, print_stats = True, min_ev = input_ev, get_total=True)
    
    return(model_score)

In [12]:
m1_score = evaluate_model(model_1, features_1, min_ev_1, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m1_score)

(4011, 4)

(4011,)

(4011, 2)

(200, 4)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,0.592835,0.407165,1
1,-105.0,-115.0,0.504097,0.495903,1
2,-130.0,110.0,0.551387,0.448613,0
3,-360.0,300.0,0.772177,0.227823,0
4,-420.0,335.0,0.806749,0.193251,0
...,...,...,...,...,...
195,-133.0,113.0,0.554838,0.445162,0
196,-210.0,175.0,0.648161,0.351839,1
197,-1100.0,700.0,0.958728,0.041272,0
198,-250.0,210.0,0.695704,0.304296,0



          Number of matches: 200
          Number of bets: 84
          Number of winning bets: 49
          Number of losing bets: 35
          Number of underdog bets: 54
          Number of underdog wins: 26
          Number of underdog losses: 28
          Number of Favorite bets: 28
          Number of favorite wins: 21
          Number of favorite losses: 7
          Number of even bets: 2
          Number of even wins: 2
          Number of even losses: 0
          Profit: 1288.3247064372893
          Profit per bet: 15.337198886158205
          Profit per match: 6.441623532186446
          
          
1288.3247064372893


In [17]:
m2_score = evaluate_model(model_2, features_2, min_ev_2, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m2_score)

(3169, 1183)

(3169,)

(3169, 2)

(200, 1183)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,0.0,1.0,1
1,-105.0,-115.0,0.0,1.0,1
2,-130.0,110.0,1.0,0.0,0
3,-360.0,300.0,1.0,0.0,0
4,-420.0,335.0,1.0,0.0,0
...,...,...,...,...,...
195,-133.0,113.0,0.0,1.0,0
196,-210.0,175.0,1.0,0.0,1
197,-1100.0,700.0,1.0,0.0,0
198,-250.0,210.0,1.0,0.0,0



          Number of matches: 200
          Number of bets: 179
          Number of winning bets: 96
          Number of losing bets: 83
          Number of underdog bets: 73
          Number of underdog wins: 31
          Number of underdog losses: 42
          Number of Favorite bets: 99
          Number of favorite wins: 63
          Number of favorite losses: 36
          Number of even bets: 7
          Number of even wins: 2
          Number of even losses: 5
          Profit: -181.73074954505586
          Profit per bet: -1.0152555840505915
          Profit per match: -0.9086537477252793
          
          
-181.73074954505586


In [14]:
m3_score = evaluate_model(model_3, features_3, min_ev_3, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m3_score)

(4011, 1367)

(4011,)

(4011, 2)

(200, 1367)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,0.295153,0.704847,1
1,-105.0,-115.0,0.122449,0.877551,1
2,-130.0,110.0,1.000000,0.000000,0
3,-360.0,300.0,1.000000,0.000000,0
4,-420.0,335.0,0.367347,0.632653,0
...,...,...,...,...,...
195,-133.0,113.0,0.897959,0.102041,0
196,-210.0,175.0,1.000000,0.000000,1
197,-1100.0,700.0,1.000000,0.000000,0
198,-250.0,210.0,0.590212,0.409788,0



          Number of matches: 200
          Number of bets: 193
          Number of winning bets: 106
          Number of losing bets: 87
          Number of underdog bets: 91
          Number of underdog wins: 38
          Number of underdog losses: 53
          Number of Favorite bets: 95
          Number of favorite wins: 65
          Number of favorite losses: 30
          Number of even bets: 7
          Number of even wins: 3
          Number of even losses: 4
          Profit: 177.3050003732361
          Profit per bet: 0.9186787584105498
          Profit per match: 0.8865250018661806
          
          
177.3050003732361


In [15]:
m4_score = evaluate_model(model_4, features_4, min_ev_4, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m4_score)

(4011, 144)

(4011,)

(4011, 2)

(200, 144)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,0.540923,0.459077,1
1,-105.0,-115.0,0.518686,0.481314,1
2,-130.0,110.0,0.554070,0.445930,0
3,-360.0,300.0,0.817896,0.182104,0
4,-420.0,335.0,0.807482,0.192518,0
...,...,...,...,...,...
195,-133.0,113.0,0.514704,0.485296,0
196,-210.0,175.0,0.651987,0.348013,1
197,-1100.0,700.0,0.905001,0.094999,0
198,-250.0,210.0,0.731412,0.268588,0



          Number of matches: 200
          Number of bets: 158
          Number of winning bets: 99
          Number of losing bets: 59
          Number of underdog bets: 87
          Number of underdog wins: 45
          Number of underdog losses: 42
          Number of Favorite bets: 66
          Number of favorite wins: 51
          Number of favorite losses: 15
          Number of even bets: 5
          Number of even wins: 3
          Number of even losses: 2
          Profit: 2436.0461461965833
          Profit per bet: 15.41801358352268
          Profit per match: 12.180230730982917
          
          
2436.0461461965833


In [16]:
m5_score = evaluate_model(model_5, features_5, min_ev_5, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

print(m5_score)

(4011, 5)

(4011,)

(4011, 2)

(200, 5)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,0.033133,0.966867,1
1,-105.0,-115.0,0.616085,0.383915,1
2,-130.0,110.0,0.725641,0.274359,0
3,-360.0,300.0,0.816348,0.183652,0
4,-420.0,335.0,0.809584,0.190416,0
...,...,...,...,...,...
195,-133.0,113.0,0.411611,0.588389,0
196,-210.0,175.0,0.686466,0.313534,1
197,-1100.0,700.0,0.996162,0.003838,0
198,-250.0,210.0,0.744149,0.255851,0



          Number of matches: 200
          Number of bets: 182
          Number of winning bets: 110
          Number of losing bets: 72
          Number of underdog bets: 67
          Number of underdog wins: 28
          Number of underdog losses: 39
          Number of Favorite bets: 110
          Number of favorite wins: 78
          Number of favorite losses: 32
          Number of even bets: 5
          Number of even wins: 4
          Number of even losses: 1
          Profit: 914.0341777479222
          Profit per bet: 5.02216581180177
          Profit per match: 4.570170888739611
          
          
914.0341777479222


In [14]:
with open('models.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
print(len(data))

5


In [16]:
data[1][1]

"DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',\n                       max_depth=None, max_features=None, max_leaf_nodes=None,\n                       min_impurity_decrease=0.0, min_impurity_split=None,\n                       min_samples_leaf=1, min_samples_split=2,\n                       min_weight_fraction_leaf=0.0, presort='deprecated',\n                       random_state=75, splitter='best')"

In [26]:
m5_score = evaluate_model(eval(data[1][1]), features_5, min_ev_5, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)

(4011, 5)

(4011,)

(4011, 2)

(200, 5)

(200,)

(200, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,-170.0,150.0,1.0,0.0,1
1,-105.0,-115.0,1.0,0.0,1
2,-130.0,110.0,1.0,0.0,0
3,-360.0,300.0,1.0,0.0,0
4,-420.0,335.0,1.0,0.0,0
...,...,...,...,...,...
195,-133.0,113.0,1.0,0.0,0
196,-210.0,175.0,1.0,0.0,1
197,-1100.0,700.0,1.0,0.0,0
198,-250.0,210.0,1.0,0.0,0



          Number of matches: 200
          Number of bets: 200
          Number of winning bets: 115
          Number of losing bets: 85
          Number of underdog bets: 64
          Number of underdog wins: 26
          Number of underdog losses: 38
          Number of Favorite bets: 129
          Number of favorite wins: 84
          Number of favorite losses: 45
          Number of even bets: 7
          Number of even wins: 5
          Number of even losses: 2
          Profit: -421.2231436940996
          Profit per bet: -2.1061157184704977
          Profit per match: -2.1061157184704977
          
          


In [24]:
model = exec(data[1][1])

In [25]:
eval(data[1][1])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')

In [22]:
print(model)

None


In [ ]:
eval